# F-test

## Import packages

In [8]:
'''
Credit to GitHub user Jaimin09
Link: https://github.com/Jaimin09/Coding-Lane-Assets/tree/main/Logistic%20Regression%20in%20Python%20from%20Scratch
Last accessed: 28/10/2021
'''
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import portablelogresmodel as model
import scipy

from scipy import stats



## Import data and set seed 

In [9]:
# ! Get dataset
filepath = 'Datasets\dec_sep_MPHWA.csv' # Data for Specialized athletes i.e. competing in one sport
df = pd.read_csv(filepath)
df = df.reset_index() # Resets index for dataset 

dec_path = 'Datasets\dec_MPHWA.csv' # Data for decathlon athletes 
dec_df = pd.read_csv(dec_path)
dec_df = dec_df.reset_index() # Resets index for dataset 

# ! Set seed and seed calling function
rng = np.random.default_rng(1) 

# List of variables we are interested in 
X_list = ['ID',  
        'PreviousMedals',
        'Height_div_avg', 
        'Weight_div_avg', 
        'Age_div_avg'
        ]

# Change X_list to exclude ID 
# X_list = X_list[1:]

# List containing medals earned for each athlete  
Y_list = ['ID', 'MedalEarned']

# Import model weights after portableregressionmodel.py has been run 
W_array = np.genfromtxt('Parameters/W.csv', delimiter=',')# Len of array is equal to the number of iterations of the model
B_array = np.genfromtxt('Parameters/B.csv', delimiter=',')

# F-test

# Wrangle data

In [10]:
# Produce weight for model
def ProduceWeights(df, W, B):

    # Get data for model
    X_model_df = df[X_list] # Dataframe contains X_list columns 
    Y_model_df = df[Y_list]

    X_array, Y_array = model.Reshape(X_model_df, Y_model_df) # Drops ID and transforms each column to a numpy array

    # Get model guesses 
    
    # Use dot product on the four weights and the variable values for each athlete i.e. Ath1 = w1*var1 + w2*var2(..) + B
    lin_func = np.dot(W.T, X_array) + B
    
    # Use linear expression in sigmoid function to get model guess for each athlete 
    sf = model.Sigmoid(lin_func) 
    
    return sf



In [11]:
# Generate model predictions in two cases. 
# 0. Model weights remain unchanged 
# 1. ith weight is set to 0
# Append model guesses to lists dependent on  the number of variables i.e. 4 variables creates a list of 4 arrays. 

def f_test_loop(W, B):
    
    # Store model guesses with weights unchanged, and create empty list to store model prediction with weights changed
   
    sig_0 = ProduceWeights(df, W, B)
    sig_1_store = []  
    
    # Exclude ID column from loop
    drop_ID = X_list[1:]
    
    for index,element in enumerate(drop_ID):
        
        # Print output for each iteration
        print(f'The current index is {index}')
        print(f'The current element is {element}')
        
        # Change weights back to values found in W
        W1 = copy.deepcopy(W)
        
        W1[index] = 0 # index weight set to 0
        
        # Get a list of model guesses where weights are changed
        sig_1 = ProduceWeights(df, W1, B)
        print(sig_1)
        
        # Append array of model guesses to list 
        sig_1_store.append(sig_1)
    
    return sig_0, sig_1_store

# Call F_test

In [12]:
# Get first weight in CSV file of weights 
W_par = np.array([W_array[0][0], W_array[1][0], W_array[2][0], W_array[3][0]], ndmin= 0)
B_par = B_array[0]

# Create list of arrays of model guesses 
p_vector_0, p_vector_1 = f_test_loop(W_par,B_par) 

The current index is 0
The current element is PreviousMedals
[0.92524216 0.20173657 0.43566097 ... 0.55253714 0.37581016 0.29995489]
The current index is 1
The current element is Height_div_avg
[0.92213526 0.19923577 0.41987367 ... 0.55507281 0.36356257 0.31676768]
The current index is 2
The current element is Weight_div_avg
[0.32566492 0.33365407 0.46003613 ... 0.46465944 0.4624237  0.44020105]
The current index is 3
The current element is Age_div_avg
[0.94741978 0.25644853 0.39402305 ... 0.48648425 0.33168873 0.25306099]


# Test for normality

In [13]:
# Test each array in sig0_store and sig1_store for normality 
# 0. Model weights remain unchanged 
# 1. ith weight is set to 0
# Append result to list 

def normality_loop(sig_0_probabilities, sig_1_probabilities):
    
    # Store normality test for model guesses with weights unchanged, i.e. sig_0
    sig_0_norm =  scipy.stats.shapiro(sig_0_probabilities)
    # Create lists to store result of normality test 
    sig_1_norm = []
    
    # ÆNDR TIL AT SMIDE UD AF X-listen!
    drop_ID = X_list[1:] # Only used for print statement 
    
    # Normality test is done for each array in the two lists of model guesses 
    for i in range(len(sig_1_probabilities)): # Ensure 
        sig_1_norm.append(scipy.stats.shapiro(sig_1_probabilities[i]))
        print(f' The current variable is: {drop_ID[i]}, sig_1_norm is currently:{sig_1_norm[i]}')
        
    return sig_0_norm,sig_1_norm

# Store normality test result for each array of weights
shapiro_0, shapiro_1 = normality_loop(p_vector_0, p_vector_1)


### THIS IS ADDTIONAL CODE TO PLOT A QQPLOT - DELETE IF NON-FUNCTIONAL BY CLOSE TO DEADLINE
#z = scipy.stats.probplot(df_sig_0["Prob"], dist = 'norm', plot = plt)
#q = scipy.stats.probplot(df_sig_1["Prob"], dist = 'norm', plot = plt)

#plt.show(z)
#plt.show(q)

# Test for normality showed that including previous medals, caused the distribution to be non-normal. PreviousMedals is therefore excluded. 

 The current variable is: PreviousMedals, sig_1_norm is currently:ShapiroResult(statistic=0.8800283074378967, pvalue=0.0)
 The current variable is: Height_div_avg, sig_1_norm is currently:ShapiroResult(statistic=0.8890291452407837, pvalue=0.0)
 The current variable is: Weight_div_avg, sig_1_norm is currently:ShapiroResult(statistic=0.8226956129074097, pvalue=0.0)
 The current variable is: Age_div_avg, sig_1_norm is currently:ShapiroResult(statistic=0.8803070187568665, pvalue=0.0)


# F-test

In [14]:
# F-test: https://link.springer.com/book/10.1007%2F978-3-319-46162-5 
# Code adapted from: https://www.statology.org/f-test-python/

# F test arrays of model guesses 
# 0. Model weights remain unchanged 
# 1. ith weight is set to 0

def f_test(sig_0_probabilities,sig_1_probabilities):
    f_test_list = []
    
    drop_ID = X_list[1:]
    
    for i in range(len(sig_1_probabilities)):
        f = np.var(sig_0_probabilities, ddof=1)/np.var(sig_1_probabilities[i], ddof=1) #calculate F test statistic 
        dfn = sig_0_probabilities.size-1 #define degrees of freedom numerator 
        dfd = sig_1_probabilities[i].size-1 #define degrees of freedom denominator 
        p = 1-scipy.stats.f.cdf(f, dfn, dfd) #find p-value of F test statistic
        f_test_list.append(p)
        
        # Print current variable and p value 
        print(f'The current variable is: {drop_ID[i]} \n The current p-value is: {p}')
        
        # Define alpha and print conclusion of hypothesis test 
        if p <= 0.05:
            print(f" null hypothesis is rejected: p-value = {p}. The probablity of the two distributions' variance being equal is low. The variable is likely to impact model output")
        else:
            print(f"null hypothesis cannot be rejected: p-value = {p}. The probablity of the two distributions' variance being equal is high. The variable is unlikely to impact model output.") 
    
    return f_test_list

f_test(p_vector_0, p_vector_1)





The current variable is: PreviousMedals 
 The current p-value is: 5.223256799169462e-05
 null hypothesis is rejected: p-value = 5.223256799169462e-05. The probablity of the two distributions' variance being equal is low. The variable is likely to impact model output
The current variable is: Height_div_avg 
 The current p-value is: 0.02825330415181948
 null hypothesis is rejected: p-value = 0.02825330415181948. The probablity of the two distributions' variance being equal is low. The variable is likely to impact model output
The current variable is: Weight_div_avg 
 The current p-value is: 1.1102230246251565e-16
 null hypothesis is rejected: p-value = 1.1102230246251565e-16. The probablity of the two distributions' variance being equal is low. The variable is likely to impact model output
The current variable is: Age_div_avg 
 The current p-value is: 0.9737840882713829
null hypothesis cannot be rejected: p-value = 0.9737840882713829. The probablity of the two distributions' variance bei

[5.223256799169462e-05,
 0.02825330415181948,
 1.1102230246251565e-16,
 0.9737840882713829]